In [187]:
from sklearn.datasets import load_iris
iris=load_iris(as_frame=True)
X=iris.data
Y= iris.target_names[iris.target]

In [188]:
from sklearn.preprocessing import add_dummy_feature, StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from copy import deepcopy

In [189]:
X_added=add_dummy_feature(X)
trainx,testx,trainy,testy= train_test_split(X,Y,test_size=0.5)
validx=testx[len(testx)//2:]
validy=testy[len(testx)//2:]
testx=testx[:len(testx)//2]
testy=testy[:len(testy)//2]
trainx_added,testx_added,trainy_added,testy_added= train_test_split(X_added,Y,test_size=0.5)
testx_added=testx_added[:len(testx_added)//2]
testy_added=testy_added[:len(testy_added)//2]
validx_added=testx_added[len(testx_added)//2:]
validy_added=testy_added[len(testx_added)//2:]

In [190]:
def totheta(model):
    theta= ([model.intercept_.tolist()])
    for i in range(len(model.coef_.reshape(4,3))):
        theta.append(model.coef_.reshape(4,3)[i].tolist())
    theta=np.array(theta)
    return theta

In [191]:
logreg=LogisticRegression()
logreg.fit(trainx,trainy)

LogisticRegression()

In [192]:
len(validx)

38

In [193]:
logreg.score(testx,testy)

0.9459459459459459

In [194]:
print(logreg.intercept_)
logreg.coef_.reshape(4,3)

[  8.3473155    2.54945867 -10.89677417]


array([[-0.42072932,  0.71718658, -1.98815348],
       [-0.90394716,  0.41535212, -0.58768576],
       [-0.21159595, -0.48336705,  0.0053772 ],
       [-0.12950082,  2.19974943,  1.38731421]])

In [195]:
ohe=OneHotEncoder(sparse_output=False)
validy_added_encoded=ohe.fit_transform(validy_added.reshape(-1,1))
testy_added_encoded=ohe.fit_transform(testy_added.reshape(-1,1))
trainy_added_encoded=ohe.fit_transform(trainy_added.reshape(-1,1))
validy_encoded=ohe.fit_transform(validy.reshape(-1,1))
testy_encoded=ohe.fit_transform(testy.reshape(-1,1))
trainy_encoded=ohe.fit_transform(trainy.reshape(-1,1))

In [196]:
error=-(logreg.predict_log_proba(validx).T@validy_encoded)/len(validx)
error

array([[0.01203973, 1.6177799 , 2.93753548],
       [0.94406704, 0.12044297, 0.81707996],
       [3.78334491, 0.83603322, 0.04287325]])

In [197]:
n_epochs=5001
m=len(validx)
scores=[]
errors=[]
best_error=np.array([[np.inf, np.inf, np.inf ],
       [np.inf , np.inf, np.inf],
       [np.inf, np.inf, np.inf]])
theta=logreg.coef_
for epoch in range(n_epochs):
   scrore=logreg.score(validx,validy)
   scores.append(scrore)
   error=-(logreg.predict_log_proba(validx).T@validy_encoded)/m
   errors.append(error)
   if (error<best_error).any():
      best_error =error
      model=deepcopy(logreg)
   gradient= ((validx.values.T@(logreg.predict_proba(validx)-validy_encoded))/m).T
   theta= theta - 0.01*gradient
   logreg.coef_=theta

In [198]:
errors

[array([[0.01203973, 1.6177799 , 2.93753548],
        [0.94406704, 0.12044297, 0.81707996],
        [3.78334491, 0.83603322, 0.04287325]]),
 array([[0.0122835 , 1.62436086, 2.93038691],
        [0.93832577, 0.11550912, 0.79901578],
        [3.78943952, 0.852873  , 0.04526543]]),
 array([[0.01249944, 1.63022936, 2.9241395 ],
        [0.9333373 , 0.11130086, 0.783206  ],
        [3.79489495, 0.86784996, 0.04747131]]),
 array([[0.0126898 , 1.63543568, 2.91867132],
        [0.92901299, 0.10770162, 0.76937182],
        [3.79978301, 0.88114474, 0.04949206]]),
 array([[0.01285682, 1.64003299, 2.91387708],
        [0.92527394, 0.10461548, 0.75726776],
        [3.80416811, 0.89292723, 0.05133266]]),
 array([[0.01300267, 1.64407491, 2.909666  ],
        [0.92205002, 0.10196321, 0.74667808],
        [3.80810789, 0.90335526, 0.0530008 ]]),
 array([[0.01312943, 1.64761382, 2.9059599 ],
        [0.91927906, 0.09967906, 0.73741352],
        [3.81165379, 0.91257406, 0.05450606]]),
 array([[0.01323902,

In [199]:
theta=totheta(logreg)

In [200]:
trainx_added@theta

array([[  3.07392251,   9.17908764, -22.13844964],
       [  2.01657724,  11.24295555, -25.36011428],
       [  3.09292801,   9.29253583, -25.92331626],
       [  3.73055362,   8.81219747, -24.9029753 ],
       [  2.49595813,  10.22771673, -24.28798515],
       [  2.60487785,   9.79358849, -26.17288   ],
       [  3.63170839,   8.5209244 , -24.73404626],
       [  1.7192142 ,  11.58520482, -27.57913489],
       [  2.59436287,   9.81783453, -26.53484083],
       [  4.5401226 ,   8.38427087, -23.31356001],
       [  2.09294203,  10.49402298, -28.93380672],
       [  2.19435995,  10.6665805 , -25.08352381],
       [  2.63062494,  10.32372288, -27.0798124 ],
       [  2.09495361,  10.97901407, -25.58377987],
       [  2.34605471,  10.51435534, -23.53022272],
       [  2.69111353,   9.67594511, -25.48319924],
       [  1.65318563,  11.93339942, -26.96774369],
       [  3.53652882,   8.44664963, -24.51358765],
       [  1.65523871,  12.39247621, -25.90301558],
       [  1.78650611,  11.65912

In [201]:
def predict_proba(theta,x):
    sigma= x@theta
    exponents= np.exp(sigma)
    return exponents/np.sum(exponents,keepdims=True,axis=1)

In [202]:
trainx_added.shape


(75, 5)

In [203]:
theta.shape

(5, 3)

In [204]:
(trainx_added.T@predict_proba(theta,trainx_added))/m

array([[3.33218042e-03, 1.97035203e+00, 2.80211298e-15],
       [1.68791113e-02, 1.16989104e+01, 1.39506491e-14],
       [9.59626567e-03, 6.07987742e+00, 7.63419724e-15],
       [7.79710271e-03, 7.75536079e+00, 8.27937155e-15],
       [1.92172004e-03, 2.54544670e+00, 2.58945044e-15]])

In [205]:
trainx.shape

(75, 4)

In [206]:
trainy.shape

(75,)

In [207]:
logreg.predict(trainx)

array(['setosa', 'setosa', 'setosa', 'versicolor', 'versicolor',
       'virginica', 'setosa', 'virginica', 'setosa', 'virginica',
       'versicolor', 'versicolor', 'setosa', 'setosa', 'setosa',
       'versicolor', 'versicolor', 'virginica', 'virginica', 'setosa',
       'virginica', 'versicolor', 'versicolor', 'virginica', 'virginica',
       'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa',
       'virginica', 'setosa', 'versicolor', 'virginica', 'setosa',
       'virginica', 'virginica', 'setosa', 'setosa', 'versicolor',
       'virginica', 'versicolor', 'virginica', 'versicolor', 'setosa',
       'versicolor', 'versicolor', 'versicolor', 'virginica',
       'versicolor', 'setosa', 'virginica', 'setosa', 'virginica',
       'setosa', 'setosa', 'versicolor', 'versicolor', 'virginica',
       'setosa', 'setosa', 'versicolor', 'versicolor', 'setosa',
       'versicolor', 'versicolor', 'virginica', 'virginica', 'setosa',
       'setosa', 'setosa', 'versicolor', 'versicolor'

In [208]:
accuracy_score(logreg.predict(trainx),trainy)

0.9466666666666667

In [209]:
n_epochs=5001
m=len(validx)
scores=[]
errors=[]
best_error=np.array([[np.inf, np.inf, np.inf ],
       [np.inf , np.inf, np.inf],
       [np.inf, np.inf, np.inf]])
theta=totheta(logreg)
for epoch in range(n_epochs):
   scrore=logreg.score(validx,validy)
   scores.append(scrore)
   error=-(logreg.predict_log_proba(validx).T@validy_encoded)/m
   errors.append(error)
   if (error<best_error).any():
      best_error =error
      model=deepcopy(logreg)
   gradient= (trainx_added.T@(predict_proba(theta,trainx_added)-trainy_added_encoded))/m
   theta= theta - 0.01*gradient
   logreg.coef_=theta

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 4)

In [ ]:
(trainx_added.T@(predict_proba(theta,trainx_added)-trainy_added_encoded))/m

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 5)